In [1]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.master("local[1]").appName("tranforms").getOrCreate()

22/12/22 17:25:59 WARN Utils: Your hostname, Magus resolves to a loopback address: 127.0.1.1; using 172.23.232.161 instead (on interface eth0)
22/12/22 17:25:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/12/22 17:26:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/22 17:26:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/22 17:26:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Prepare Data
simpleData = (("Java",4000,5), \
    ("Python", 4600,10),  \
    ("Scala", 4100,15),   \
    ("Scala", 4500,15),   \
    ("PHP", 3000,20),  \
  )
columns= ["CourseName", "fee", "discount"]

# Create DataFrame
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- CourseName: string (nullable = true)
 |-- fee: long (nullable = true)
 |-- discount: long (nullable = true)



+----------+----+--------+
|CourseName|fee |discount|
+----------+----+--------+
|Java      |4000|5       |
|Python    |4600|10      |
|Scala     |4100|15      |
|Scala     |4500|15      |
|PHP       |3000|20      |
+----------+----+--------+



## DataFrame.transform()

used to chain the custom transformations and this function returns new DF after applying transformations


In [10]:
# Create cusotom transformations

from pyspark.sql.functions import upper
import pyspark.sql

# Custom transformation 1
def to_upper_str_cols(df: pyspark.sql.DataFrame):
    return df.withColumn("CourseName", upper(df.CourseName))

# Custom transformation 2
def reduce_price(reduceBy: int):
    def inner(df: pyspark.sql.DataFrame):
        return df.withColumn("new_fee",df.fee - reduceBy)
    
    return inner


# Custom transformation 3
def apply_discount(df: pyspark.sql.DataFrame):
    return df.withColumn("discounted_fee",  \
             df.new_fee - (df.new_fee * df.discount) / 100)

In [17]:
df2 = df.transform(to_upper_str_cols) \
        .transform(reduce_price(1000)) \
        .transform(apply_discount) \
        .show()

+----------+----+--------+-------+--------------+
|CourseName| fee|discount|new_fee|discounted_fee|
+----------+----+--------+-------+--------------+
|      JAVA|4000|       5|   3000|        2850.0|
|    PYTHON|4600|      10|   3600|        3240.0|
|     SCALA|4100|      15|   3100|        2635.0|
|     SCALA|4500|      15|   3500|        2975.0|
|       PHP|3000|      20|   2000|        1600.0|
+----------+----+--------+-------+--------------+



## sql.functions.transform()

used to apply transformation on column of type Array

In [18]:
data = [
 ("James,,Smith",["Java","Scala","C++"],["Spark","Java"]),
 ("Michael,Rose,",["Spark","Java","C++"],["Spark","Java"]),
 ("Robert,,Williams",["CSharp","VB"],["Spark","Python"])
]
df = spark.createDataFrame(data=data,schema=["Name","Languages1","Languages2"])
df.printSchema()
df.show()

root
 |-- Name: string (nullable = true)
 |-- Languages1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Languages2: array (nullable = true)
 |    |-- element: string (containsNull = true)

+----------------+------------------+---------------+
|            Name|        Languages1|     Languages2|
+----------------+------------------+---------------+
|    James,,Smith|[Java, Scala, C++]|  [Spark, Java]|
|   Michael,Rose,|[Spark, Java, C++]|  [Spark, Java]|
|Robert,,Williams|      [CSharp, VB]|[Spark, Python]|
+----------------+------------------+---------------+



In [19]:
from pyspark.sql.functions import upper
from pyspark.sql.functions import transform
df.select(transform("Languages1", lambda x: upper(x)).alias("languages1")) \
  .show()

+------------------+
|        languages1|
+------------------+
|[JAVA, SCALA, C++]|
|[SPARK, JAVA, C++]|
|      [CSHARP, VB]|
+------------------+

